In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib
import os
from IPython.display import display
matplotlib.use('Agg')
%matplotlib inline
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from gather_metrics import *
def get_ranked_by_means_df(gather_df, metric_list):
    means_df = gather_df.xs(['test_metrics', 'Folds Mean'], level=['set','fold'])
    ranked_df = pd.DataFrame(data=np.zeros(shape=means_df.shape), columns=metric_list, dtype=str)
    for m, i in zip(metric_list, range(len(metric_list))):
        metric_df = means_df[m]
        sorted_df = metric_df.sort_values(ascending=False)
        models_ranked = sorted_df.index.tolist()
        ranked_df.iloc[:,i] = models_ranked
    return ranked_df

# load stage PS
model_directory = '../../../job_results_small/'
class_dirs = [model_directory+'/random_forest/stage_2/',
              model_directory+'/irv/stage_2/',
              model_directory+'/neural_networks/stage_2/',
              model_directory+'/docking/stage_2/',
              model_directory+'/baseline/stage_2/']
gather_df_3 = gather_dir_metrics(class_dirs[0], 1)     
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[1], 1))   
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[2], 1))   
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[3], 1))
gather_df_3 = gather_df_3.append(gather_dir_metrics(class_dirs[4], 1))

col_indices = list(range(10)) + list(range(15, 20)) + list(range(25, 105)) + list(range(145, 149)) + list(range(150, 183))
col_indices = [i for i in col_indices if 'Mean' not in gather_df_3.columns[i] and 'Median' not in gather_df_3.columns[i]]
col_indices = [i for i in col_indices if 'PriA-SSB AS' in gather_df_3.columns[i]]
metric_names = list(gather_df_3.columns.values[col_indices])

gather_df_3 = gather_df_3[metric_names]
gather_df_3 = gather_df_3.dropna(axis=0)

gather_df_ps = gather_df_3

In [2]:
# load stage CV
model_directory = '../../../job_results_small/'
class_dirs = [model_directory+'/random_forest/stage_1/',
              model_directory+'/irv/stage_1/',
              model_directory+'/neural_networks/stage_1/',
              model_directory+'/docking/stage_1/']
k = 5
gather_df_2 = gather_dir_metrics(class_dirs[0], k)     
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[1], k))   
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[2], 20))
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[3], k))
gather_df_2 = gather_df_2[metric_names]
gather_df_2 = gather_df_2.dropna(axis=0)

gather_df_cv = gather_df_2

#setup comparison matrix and overlaps
dtk_dict = dtk_multi_metrics(gather_df_cv, list(range(len(col_indices))))
dtk_analysis_dict = analyze_dtk_dict(dtk_dict)
mm_comp_dicts = get_mean_median_comps(gather_df_cv, list(range(len(col_indices))))
agg_comp_dict = get_agg_comp(mm_comp_dicts, dtk_analysis_dict, mean_w=1, median_w=0, tukey_w=1)

In [3]:
ordered_df = get_model_ordering(agg_comp_dict, metric_names)
ordered_df

,ROC AUC PriA-SSB AS,BEDROC AUC PriA-SSB AS,PR auc.integral PriA-SSB AS,NEF_0.1 % PriA-SSB AS,NEF_0.15 % PriA-SSB AS,NEF_0.5 % PriA-SSB AS,NEF_1 % PriA-SSB AS,NEF_2 % PriA-SSB AS,NEF_5 % PriA-SSB AS,NEF_10 % PriA-SSB AS,NEF_20 % PriA-SSB AS,EF_0.1 % PriA-SSB AS,EF_0.15 % PriA-SSB AS,EF_0.5 % PriA-SSB AS,EF_1 % PriA-SSB AS,EF_2 % PriA-SSB AS,EF_5 % PriA-SSB AS,EF_10 % PriA-SSB AS,EF_20 % PriA-SSB AS,NEF AUC PriA-SSB AS,n_hits_100 PriA-SSB AS,n_hits_250 PriA-SSB AS,n_hits_500 PriA-SSB AS,n_hits_1000 PriA-SSB AS,n_hits_2500 PriA-SSB AS,n_hits_5000 PriA-SSB AS,n_hits_10000 PriA-SSB AS
0,"RandomForest_d, 1.0","RandomForest_h, 1.0","SingleClassification_b, 1.0","RandomForest_g, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_g, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_c, 1.0","RandomForest_d, 1.0","IRV_e, 1.0"
1,"RandomForest_e, 2.0","RandomForest_g, 2.0","SingleClassification_a, 2.0","RandomForest_a, 2.0","RandomForest_b, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 1.0","RandomForest_d, 1.0","RandomForest_c, 1.0","RandomForest_h, 2.0","RandomForest_b, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 1.0","RandomForest_d, 1.0","RandomForest_c, 1.0","RandomForest_g, 2.0","RandomForest_e, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_d, 1.0","RandomForest_b, 1.0","RandomForest_g, 2.0","IRV_d, 2.0"
2,"RandomForest_h, 3.0","RandomForest_d, 3.0","MultiClassification_a, 3.0","RandomForest_b, 2.0","RandomForest_c, 2.0","RandomForest_a, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0","RandomForest_e, 3.0","RandomForest_g, 3.0","RandomForest_b, 1.0","RandomForest_a, 3.0","RandomForest_c, 2.0","RandomForest_a, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0","RandomForest_e, 3.0","RandomForest_g, 3.0","RandomForest_b, 1.0","RandomForest_d, 3.0","RandomForest_g, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0","RandomForest_g, 3.0","RandomForest_a, 1.0","SingleRegression_b, 3.0","IRV_a, 2.0"
3,"RandomForest_g, 4.0","RandomForest_e, 4.0","RandomForest_g, 4.0","RandomForest_h, 4.0","RandomForest_g, 4.0","RandomForest_d, 4.0","RandomForest_e, 3.0","RandomForest_e, 3.0","RandomForest_d, 3.0","RandomForest_e, 3.0","RandomForest_a, 1.0","RandomForest_b, 3.0","RandomForest_g, 4.0","RandomForest_d, 4.0","RandomForest_e, 3.0","RandomForest_e, 3.0","RandomForest_d, 3.0","RandomForest_e, 3.0","RandomForest_a, 1.0","RandomForest_e, 4.0","RandomForest_d, 4.0","RandomForest_e, 4.0","RandomForest_e, 4.0","RandomForest_e, 3.0","SingleRegression_b, 4.0","RandomForest_c, 3.0","IRV_b, 2.0"
4,"RandomForest_c, 5.0","RandomForest_c, 5.0","RandomForest_h, 5.0","RandomForest_d, 5.0","IRV_e, 5.0","RandomForest_c, 4.0","RandomForest_a, 5.0","IRV_e, 5.0","RandomForest_c, 5.0","RandomForest_b, 3.0","SingleRegression_b, 5.0","RandomForest_c, 5.0","IRV_e, 5.0","RandomForest_c, 4.0","RandomForest_a, 5.0","IRV_e, 5.0","RandomForest_c, 5.0","RandomForest_b, 3.0","SingleRegression_b, 5.0","RandomForest_b, 5.0","RandomForest_c, 5.0","RandomForest_b, 5.0","RandomForest_c, 5.0","RandomForest_c, 5.0","RandomForest_h, 4.0","RandomForest_h, 5.0","IRV_c, 2.0"
5,"RandomForest_a, 6.0","RandomForest_b, 6.0","RandomForest_e, 6.0","RandomForest_c, 5.0","RandomForest_e, 6.0","RandomForest_b, 4.0","RandomForest_b, 6.0","RandomForest_c, 6.0","RandomForest_b, 5.0","RandomForest_c, 6.0","RandomForest_g, 6.0","RandomForest_d, 6.0","RandomForest_e, 6.0","RandomForest_b, 4.0","RandomForest_b, 6.0","RandomForest_c, 6.0","RandomForest_b, 5.0","RandomForest_c, 6.0","RandomForest_g, 6.0","RandomForest_c, 6.0","RandomForest_b, 5.0","RandomFor

---
## Compute Best Models for Each Metric Based on Mean VS DTK+Mean

In [4]:
ps_250_hits = gather_df_ps.xs(['test_metrics', 'fold 0'], level=['set','fold'])['n_hits_250 PriA-SSB AS']
simple_ranked_cv = get_ranked_by_means_df(gather_df_cv, metric_names)
complex_ranked_cv = get_model_ordering(agg_comp_dict, metric_names)
oracle_pick = ps_250_hits.sort_values(ascending=False).iloc[0]

top_n = 1
req_metrics = ['ROC AUC PriA-SSB AS', 'BEDROC AUC PriA-SSB AS','PR auc.integral PriA-SSB AS', 
               'NEF AUC PriA-SSB AS', 'NEF_1 % PriA-SSB AS', 'EF_1 % PriA-SSB AS']

simple_top_n = simple_ranked_cv[req_metrics].iloc[:top_n,:]
complex_top_n = complex_ranked_cv[req_metrics].iloc[:top_n,:]
display('Mean Top 5 on CV-Simple', simple_ranked_cv[req_metrics].iloc[:41,:])
display('Mean Top 5 on CV-Complex', complex_ranked_cv[req_metrics].iloc[:41,:])

'Mean Top 5 on CV-Simple'

,ROC AUC PriA-SSB AS,BEDROC AUC PriA-SSB AS,PR auc.integral PriA-SSB AS,NEF AUC PriA-SSB AS,NEF_1 % PriA-SSB AS,EF_1 % PriA-SSB AS
0,RandomForest_d,RandomForest_h,RandomForest_g,RandomForest_h,RandomForest_h,RandomForest_h
1,RandomForest_e,RandomForest_g,RandomForest_h,RandomForest_g,RandomForest_g,RandomForest_g
2,RandomForest_h,RandomForest_d,RandomForest_e,RandomForest_d,RandomForest_d,RandomForest_d
3,RandomForest_g,RandomForest_e,RandomForest_d,RandomForest_e,RandomForest_e,RandomForest_e
4,RandomForest_c,RandomForest_c,RandomForest_b,RandomForest_b,RandomForest_a,RandomForest_a
5,RandomForest_a,RandomForest_b,RandomForest_c,RandomForest_c,RandomForest_c,RandomForest_c
6,RandomForest_b,RandomForest_a,RandomForest_a,RandomForest_a,RandomForest_b,RandomForest_b
7,SingleRegression_b,IRV_e,IRV_d,IRV_e,IRV_c,IRV_c
8,IRV_e,IRV_d,IRV_b,RandomForest_f,IRV_d,IRV_d
9,MultiClassification_b,RandomForest_f,IRV_c,IRV_d,IRV_b,IRV_b


'Mean Top 5 on CV-Complex'

,ROC AUC PriA-SSB AS,BEDROC AUC PriA-SSB AS,PR auc.integral PriA-SSB AS,NEF AUC PriA-SSB AS,NEF_1 % PriA-SSB AS,EF_1 % PriA-SSB AS
0,"RandomForest_d, 1.0","RandomForest_h, 1.0","SingleClassification_b, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0"
1,"RandomForest_e, 2.0","RandomForest_g, 2.0","SingleClassification_a, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0"
2,"RandomForest_h, 3.0","RandomForest_d, 3.0","MultiClassification_a, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0"
3,"RandomForest_g, 4.0","RandomForest_e, 4.0","RandomForest_g, 4.0","RandomForest_e, 4.0","RandomForest_e, 3.0","RandomForest_e, 3.0"
4,"RandomForest_c, 5.0","RandomForest_c, 5.0","RandomForest_h, 5.0","RandomForest_b, 5.0","RandomForest_a, 5.0","RandomForest_a, 5.0"
5,"RandomForest_a, 6.0","RandomForest_b, 6.0","RandomForest_e, 6.0","RandomForest_c, 6.0","RandomForest_b, 6.0","RandomForest_b, 6.0"
6,"RandomForest_b, 7.0","RandomForest_a, 7.0","RandomForest_d, 7.0","RandomForest_a, 7.0","RandomForest_c, 6.0","RandomForest_c, 6.0"
7,"SingleRegression_b, 8.0","IRV_e, 8.0","RandomForest_b, 8.0","IRV_e, 8.0","IRV_c, 8.0","IRV_c, 8.0"
8,"MultiClassification_b, 9.0","IRV_d, 9.0","RandomForest_c, 9.0","RandomForest_f, 9.0","IRV_d, 8.0","IRV_d, 8.0"
9,"IRV_e, 10.0","RandomForest_f, 10.0","RandomForest_a, 10.0","IRV_d, 10.0","IRV_b, 10.0","IRV_b, 10.0"


In [5]:
simple_regret = simple_top_n.copy()
complex_regret = complex_top_n.copy()

for m in req_metrics:
    simple_model = simple_regret[m].iloc[0]
    complex_model = complex_regret[m].iloc[0]
    complex_model = complex_model[:complex_model.index(',')]
    complex_top_n[m].iloc[0] = complex_model
    
    simple_regret.loc[0,m] = oracle_pick - ps_250_hits.loc[simple_model]
    complex_regret.loc[0,m] = oracle_pick - ps_250_hits.loc[complex_model]

In [6]:
final_df = pd.merge(simple_top_n.T, simple_regret.T, left_index=True, right_index=True)
final_df = pd.merge(final_df, complex_top_n.T, left_index=True, right_index=True)
final_df = pd.merge(final_df, complex_regret.T, left_index=True, right_index=True)
columns = [('Best by Mean','Model'),('Best by Mean','Regret'),('Best by DTK','Model'),('Best by DTK','Regret')]
final_df.columns=pd.MultiIndex.from_tuples(columns)
final_df.columns.name = 'Metric'
final_df.index = [s.replace(' PriA-SSB AS', '') for s in final_df.index]

display(final_df)

Best by Mean                    Best by DTK       
                          Model Regret                   Model Regret
ROC AUC          RandomForest_d      3          RandomForest_d      3
BEDROC AUC       RandomForest_h      0          RandomForest_h      0
PR auc.integral  RandomForest_g      1  SingleClassification_b      7
NEF AUC          RandomForest_h      0          RandomForest_h      0
NEF_1 %          RandomForest_h      0          RandomForest_h      0
EF_1 %           RandomForest_h      0          RandomForest_h      0

In [7]:
display('DTK+Mean', final_df)

'DTK+Mean'

Best by Mean                    Best by DTK       
                          Model Regret                   Model Regret
ROC AUC          RandomForest_d      3          RandomForest_d      3
BEDROC AUC       RandomForest_h      0          RandomForest_h      0
PR auc.integral  RandomForest_g      1  SingleClassification_b      7
NEF AUC          RandomForest_h      0          RandomForest_h      0
NEF_1 %          RandomForest_h      0          RandomForest_h      0
EF_1 %           RandomForest_h      0          RandomForest_h      0